# Testing SAP-RPT-1 on the Titanic challenge

In this notebook I'm going to walk you through my experiments working on the [Titanic challenge from Kaggle](https://www.kaggle.com/competitions/titanic) with [SAP-RPT-1](https://community.sap.com/t5/artificial-intelligence-blogs-posts/building-an-sap-rpt-1-quot-hello-world-quot-with-ai-core/ba-p/14292100).

Here is the outline of this notebook:

- First we're going to create some helper functions for making predictions with RPT-1.
- Since I wanted to run quite a few experiments on the Titanic challenge, I thought it would be a good idea to only work on the training data and mask some entries so that I can judge model performance without submitting to Kaggle. This produced some surprising results that we're going to explore.
- We're going to pivot to working on the full data set and submitting the results to Kaggle.
- Throughout this notebook, we're going to create Gradient Boosting models to benchmark RPT-1's performance.

> Note: If you are looking for a more business-oriented summary, please check out my related blog post on [chrwittm.github.io](https://chrwittm.github.io/posts/2026-01-09-lets-test-rpt-1-on-titanic/) or [SAP Communities](https://community.sap.com/t5/artificial-intelligence-blogs-posts/let-s-test-rpt-1-on-titanic/ba-p/14304233).

## Creating helper functions

This section contains helper functions for working and making predictions with RPT-1. (See the [Hello World blog post](https://community.sap.com/t5/artificial-intelligence-blogs-posts/building-an-sap-rpt-1-quot-hello-world-quot-with-ai-core/ba-p/14292100) for details on setting up SAP AI Core and authentication.)

In [88]:
# Choose your model size

model_size = "large"
#model_size = "small"

model = f"sap-rpt-1-{model_size}"
model

'sap-rpt-1-large'

In [89]:
_cached_token = None

def get_bearer_token(force_refresh=False):
    """Get bearer token with simple caching."""
    global _cached_token
    
    if _cached_token and not force_refresh:
        #print("Using cached bearer token.")
        return _cached_token
    
    # Fetch new token
    #print("Fetching new bearer token...")
    
    AICORE_AUTH_URL = os.environ["AICORE_AUTH_URL"]
    AICORE_CLIENT_ID = os.environ["AICORE_CLIENT_ID"]
    AICORE_CLIENT_SECRET = os.environ["AICORE_CLIENT_SECRET"]

    token_url = f"{AICORE_AUTH_URL}/oauth/token"
    
    response = requests.post(
        token_url,
        auth=(AICORE_CLIENT_ID, AICORE_CLIENT_SECRET),
        data={"grant_type": "client_credentials"},
    )
    
    response.raise_for_status()
    
    token_data = response.json()
    _cached_token = token_data["access_token"]
    
    return _cached_token

In [90]:
def get_deployment_url(model):
    AICORE_API_URL = os.environ["AICORE_API_URL"].rstrip("/")

    model_to_env_var = {
        "sap-rpt-1-large": "RPT1L_DEPLOYMENT_ID",
        "sap-rpt-1-small": "RPT1S_DEPLOYMENT_ID"
    }
    
    if model not in model_to_env_var:
        raise ValueError(f"Unknown model: {model}. Choose from {list(model_to_env_var.keys())}")
    
    env_var_name = model_to_env_var[model]
    deployment_id = os.environ.get(env_var_name)

    if not deployment_id:
        raise ValueError(f"Missing {env_var_name} in .env (deployment id from AI Launchpad).")

    return f"{AICORE_API_URL}/inference/deployments/{deployment_id}/predict"

In [91]:
def get_headers():

    AICORE_RESOURCE_GROUP = os.environ.get("AICORE_RESOURCE_GROUP", "default")
    access_token = get_bearer_token()

    headers = {
        "Authorization": f"Bearer {access_token}",
        "AI-Resource-Group": AICORE_RESOURCE_GROUP,
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    return headers

In [92]:
import time

def call_rpt(payload, model, max_retries=7):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                get_deployment_url(model), 
                headers=get_headers(), 
                json=payload, 
                timeout=120
            )
            response.raise_for_status()
            return response.json()
            
        except requests.HTTPError as e:
            if e.response.status_code == 429:  # Rate limit
                wait_time = 2 ** attempt  # Exponential backoff: 1, 2, 4, 8, 16 seconds
                print(f"Rate limited. Waiting {wait_time} seconds before retry {attempt + 1}/{max_retries}...")
                time.sleep(wait_time)
            else:
                print(f"Status code: {e.response.status_code}")
                print(f"Reason: {e.response.reason}")
                print(f"Response text: {e.response.text}")
                raise
    
    raise Exception(f"Failed after {max_retries} retries due to rate limiting")

## Loading the Titanic Dataset

In the section, we're going to load the Titanic data from Kaggle.

In [93]:
#!pip install kaggle

In [94]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()
os.environ["KAGGLE_API_TOKEN"] = os.getenv("KAGGLE_API_TOKEN")


In [95]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [96]:
api.competition_download_files("titanic", path=".", quiet=False)

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [97]:
# Upzip the downloaded data if you're running this for the first time
#!unzip -q titanic.zip

In [98]:
import pandas as pd

training_data = pd.read_csv("train.csv")
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:
training_data["Survived"] = training_data["Survived"].astype(str) # for masking with "[PREDICT]"

## Converting the Dataframe to a RPT-1 compatible payload

As we have seen in the hello world example, we need to present the data to RPT-1 in a certain format as specified [in the documentation](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/example-payloads-for-inferencing-sap-rpt-1).

To show the structure of the data we sent to RPT-1 let's use an minimal set of just 5 lines.

In [100]:
mini_training_data = training_data[:5].copy()
mini_training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [101]:
def get_data_schema(df):
    data_schema = {}
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            dtype = "date"
        elif df[col].dtype == 'object':
            dtype = "string"
        else:
            dtype = "numeric"
        data_schema[col] = {"dtype": dtype}
    return data_schema

In [102]:
get_data_schema(mini_training_data)

{'PassengerId': {'dtype': 'numeric'},
 'Survived': {'dtype': 'string'},
 'Pclass': {'dtype': 'numeric'},
 'Name': {'dtype': 'string'},
 'Sex': {'dtype': 'string'},
 'Age': {'dtype': 'numeric'},
 'SibSp': {'dtype': 'numeric'},
 'Parch': {'dtype': 'numeric'},
 'Ticket': {'dtype': 'string'},
 'Fare': {'dtype': 'numeric'},
 'Cabin': {'dtype': 'string'},
 'Embarked': {'dtype': 'string'}}

In [103]:
def mask_entry(df, passenger_id):
    """Replaces the target value with [PREDICT] token for inference"""
    df.loc[df["PassengerId"] == passenger_id, "Survived"] = '[PREDICT]'
    return df

In [104]:
mini_training_data = mask_entry(mini_training_data, 5)
mini_training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,[PREDICT],3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [105]:
def get_rpt_payload(df, target_column="Survived", index_column="PassengerId", task_type="classification"):
    """
    Converts a pandas DataFrame to RPT-1 API payload format.
    
    Args:
        df: pandas DataFrame
        target_column: name of the column to predict
        index_column: name of the column to use as row index
        task_type: "classification" or "regression"
    
    Returns:
        dict: payload ready for RPT-1 API
    """
    
    df_copy = df.copy()
    
    # Convert categorical columns to strings before filling
    for col in df_copy.columns:
        if df_copy[col].dtype.name == 'category':
            df_copy[col] = df_copy[col].astype(str)
    
    df_copy = df_copy.fillna("")  # JSON does not like null values
    
    rows = df_copy.to_dict('records')
    
    payload = {
        "prediction_config": {
            "target_columns": [
                {
                    "name": target_column,
                    "prediction_placeholder": "[PREDICT]",
                    "task_type": task_type
                }
            ]
        },
        "index_column": index_column,
        "rows": rows,
        "data_schema": get_data_schema(df)
    }
    
    return payload

In [106]:
payload = get_rpt_payload(mini_training_data)
payload

{'prediction_config': {'target_columns': [{'name': 'Survived',
    'prediction_placeholder': '[PREDICT]',
    'task_type': 'classification'}]},
 'index_column': 'PassengerId',
 'rows': [{'PassengerId': 1,
   'Survived': '0',
   'Pclass': 3,
   'Name': 'Braund, Mr. Owen Harris',
   'Sex': 'male',
   'Age': 22.0,
   'SibSp': 1,
   'Parch': 0,
   'Ticket': 'A/5 21171',
   'Fare': 7.25,
   'Cabin': '',
   'Embarked': 'S'},
  {'PassengerId': 2,
   'Survived': '1',
   'Pclass': 1,
   'Name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
   'Sex': 'female',
   'Age': 38.0,
   'SibSp': 1,
   'Parch': 0,
   'Ticket': 'PC 17599',
   'Fare': 71.2833,
   'Cabin': 'C85',
   'Embarked': 'C'},
  {'PassengerId': 3,
   'Survived': '1',
   'Pclass': 3,
   'Name': 'Heikkinen, Miss. Laina',
   'Sex': 'female',
   'Age': 26.0,
   'SibSp': 0,
   'Parch': 0,
   'Ticket': 'STON/O2. 3101282',
   'Fare': 7.925,
   'Cabin': '',
   'Embarked': 'S'},
  {'PassengerId': 4,
   'Survived': '1',
   'Pclass': 1

## Making the First Prediction

Let's put everything together and do a first prediction with RPT-1.

In [107]:
def get_rpt_predictions(df, model):
    payload = get_rpt_payload(df)
    data = call_rpt(payload, model)
    return data["predictions"]

In [108]:
preds = get_rpt_predictions(mini_training_data, model)
print(json.dumps(preds, indent=2, ensure_ascii=False))

[
  {
    "PassengerId": 5,
    "Survived": [
      {
        "confidence": 0.83,
        "prediction": "1"
      }
    ]
  }
]


In [109]:
# The actual value was:
training_data.loc[training_data["PassengerId"] == 5, "Survived"].values[0]

'0'

RPT-1 predicts survival with 83% confidence, but passenger 5 actually didn't survive. With only 4 training examples, this isn't too surprising, after all the model doesn't have much context to work with. Let's see how it performs with the full training set.

## Running Predictions on the Whole Training Set

Now let's scale this up. We'll use the same approach but with the full 891 training examples, still masking just the last 5 entries to see how RPT-1 performs.

Along the way, we are also going to setup some more scalable functions for additional experiments with more data. Depending on the model size, you can only make a certain number of predictions during one inference call.

In [110]:
def mask_last_entries(n, df, column_name = 'Survived', mask='[PREDICT]'):
    masked_df = df.copy()
    masked_df.loc[masked_df.index[-n:], column_name] = mask
    return masked_df

In [111]:
masked_training_data = mask_last_entries(5, training_data)
masked_training_data.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,[PREDICT],2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,[PREDICT],1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,[PREDICT],3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,[PREDICT],1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,[PREDICT],3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [112]:
def separate_masked_unmasked(df, column_name = 'Survived', mask='[PREDICT]'):
    masked_entries = df[df[column_name] == mask]
    unmasked_entries = df[df[column_name] != mask]
    return masked_entries, unmasked_entries

In [113]:
masked_df, unmasked_df = separate_masked_unmasked(masked_training_data)
print(len(training_data))
print(len(masked_df))
print(len(unmasked_df))

891
5
886


In [114]:
def get_rpt_predictions(df, model, column_name='Survived', batch_size=25):
    """Returns RPT-1 predictions splitting the data into batches
    Batch size 25 was my testing limit, keeping it for the time being to keep things re-produceable.
    """

    preds = []
    masked_df, unmasked_df = separate_masked_unmasked(df, column_name)

    for i in range(0,len(masked_df), batch_size):
        start = i
        end = i+batch_size if i+batch_size < len(masked_df) else len(masked_df)
        combined_df = pd.concat([unmasked_df, masked_df[start:end]])
        payload = get_rpt_payload(combined_df, target_column=column_name)
        data = call_rpt(payload, model)
        preds += data["predictions"]

    return preds

In [115]:
preds = get_rpt_predictions(masked_training_data, model)
preds

[{'PassengerId': 887, 'Survived': [{'confidence': 0.97, 'prediction': '0'}]},
 {'PassengerId': 888, 'Survived': [{'confidence': 1.0, 'prediction': '1'}]},
 {'PassengerId': 889, 'Survived': [{'confidence': 1.0, 'prediction': '0'}]},
 {'PassengerId': 890, 'Survived': [{'confidence': 1.0, 'prediction': '1'}]},
 {'PassengerId': 891, 'Survived': [{'confidence': 1.0, 'prediction': '0'}]}]

In [116]:
# Show predictions vs actual
for pred in preds:
    pid = pred['PassengerId']
    actual = training_data.loc[training_data['PassengerId'] == pid, 'Survived'].values[0]
    predicted = pred['Survived'][0]['prediction']
    confidence = pred['Survived'][0]['confidence']
    print(f"PassengerId {pid}: Predicted={predicted}, Actual={actual}, Confidence={confidence}")

PassengerId 887: Predicted=0, Actual=0, Confidence=0.97
PassengerId 888: Predicted=1, Actual=1, Confidence=1.0
PassengerId 889: Predicted=0, Actual=0, Confidence=1.0
PassengerId 890: Predicted=1, Actual=1, Confidence=1.0
PassengerId 891: Predicted=0, Actual=0, Confidence=1.0


In [117]:
def calculate_accuracy(original_df, masked_df, predictions, target_column='Survived'):
    """Calculate accuracy of predictions on masked rows."""
    df = original_df.copy()
    target_column_masked = f"{target_column}_Masked"
    target_column_pred = f"{target_column}_Pred"
    df[target_column_masked] = masked_df[target_column]
    
    # Add predictions to dataframe
    for pred in predictions:
        passenger_id = pred['PassengerId']
        predicted_value = pred[target_column][0]['prediction']
        df.loc[df['PassengerId'] == passenger_id, target_column_pred] = predicted_value
    
    # Calculate accuracy only on predicted rows
    predicted_rows = df[df[target_column_masked] == '[PREDICT]']
    accuracy = (predicted_rows[target_column] == predicted_rows[target_column_pred]).mean()
    
    return accuracy

In [118]:
calculate_accuracy(training_data, masked_training_data, preds)

np.float64(1.0)

In this set up, with 886 lines in the context window, RPT-1 correctly predicted the masked five entries - good job!

But this perfect score raises an interesting question: What happens when we mask more entries?

## Running Predictions on More Masked Data

Let's scale things up even further and push things to the limit by masking a lot more entries.

For a start, let's mask about half of the data, the last 400 entries.

> Note: You may see rate limiting messages - this is normal when making many API calls

In [119]:
masked_training_data = mask_last_entries(400, training_data)
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
0.9825


Wow, an almost perfect score, almost too good to be true. Let's try a variation with random masking.

In [120]:
import random

def mask_random_entries(n, df, column_name = 'Survived', mask='[PREDICT]'):
    masked_df = df.copy()
    random.seed(42)
    
    # Get n unique random indices
    indices = random.sample(range(len(df)), n)
    
    for df_index in indices:
        masked_df.loc[masked_df.index[df_index], column_name] = mask
    
    return masked_df

In [121]:
random_masked = mask_random_entries(400, training_data)
random_preds = get_rpt_predictions(random_masked,model)
print(calculate_accuracy(training_data, random_masked, random_preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
0.9725


Almost the same results. This looks very suspicious. Just as a quick reminder for the baseline: If you just predict that all women survive and all men died, your accuracy will be about 76%. Anything above 80% is super competitive, and there's no serious submission on Kaggle with more than 90%.

My first thought was a masking bug, but I've verified the masking logic is correct.

Let's push this further with even more masked entries to understand what's really happening.

## Masking More Data

In [122]:
random_masked = mask_random_entries(800, training_data)
random_preds = get_rpt_predictions(random_masked, model)
print(calculate_accuracy(training_data, random_masked, random_preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
0.97375


This is definitively odd. Could it be that RPT-1 knows the Titanic data set and just regurgitates the results?

Let's try to find out if RPT-1 remembers the data set from training by making it predict passenger names.

## Predicting Passenger Names

If RPT-1 is achieving 98% accuracy by memorizing the Titanic dataset from training, it should also be able to predict passenger names. Let's test this hypothesis.

In [123]:
random_masked = mask_random_entries(400, training_data, column_name='Name')
random_preds = get_rpt_predictions(random_masked, model, column_name='Name')
print(calculate_accuracy(training_data, random_masked, random_preds, target_column='Name'))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
0.0


This is almost surprising, but reassuring: RPT-1 was not able to predict any name, which means it did not remember that they just sent from training. But what did it actually predict?

In [124]:
for p in random_preds[:5]:
    pid = p['PassengerId']
    actual = training_data[training_data['PassengerId'] == pid]['Name'].values[0]
    pred = p['Name'][0]['prediction']
    print(f"PID {pid}:")
    print(f"  Actual: {actual}")
    print(f"  Pred:   {pred}")
    print(f"  Confidence: {p['Name'][0]['confidence']}")
    print()

PID 1:
  Actual: Braund, Mr. Owen Harris
  Pred:   Nye, Mrs. (Elizabeth Ramell)
  Confidence: 0.0

PID 3:
  Actual: Heikkinen, Miss. Laina
  Pred:   Beavan, Mr. William Thomas
  Confidence: 0.0

PID 4:
  Actual: Futrelle, Mrs. Jacques Heath (Lily May Peel)
  Pred:   Jussila, Miss. Katriina
  Confidence: 0.01

PID 7:
  Actual: McCarthy, Mr. Timothy J
  Pred:   Bateman, Rev. Robert James
  Confidence: 0.0

PID 8:
  Actual: Palsson, Master. Gosta Leonard
  Pred:   Palsson, Miss. Stina Viola
  Confidence: 0.07



These names look real, but where did they come from?

In [125]:
# Get the unmasked names that were sent as context
masked_df, unmasked_df = separate_masked_unmasked(random_masked, column_name='Name')
context_names = set(unmasked_df['Name'].values)

# Check if predicted names are from the context
predicted_names = [p['Name'][0]['prediction'] for p in random_preds]
from_context = [name for name in predicted_names if name in context_names]
not_from_context = [name for name in predicted_names if name not in context_names]

print(f"Total predictions: {len(predicted_names)}")
print(f"From context (unmasked rows): {len(from_context)}")
print(f"NOT from context: {len(not_from_context)}")
print(f"\nPercentage from context: {len(from_context)/len(predicted_names)*100:.1f}%")

if not_from_context[:5]:
    print(f"\nExample names NOT in context:")
    for name in not_from_context[:5]:
        print(f"  - {name}")

Total predictions: 400
From context (unmasked rows): 400
NOT from context: 0

Percentage from context: 100.0%


As it turns out, RPT-1 is repeating names from the unmasked data in its context window at very low confidence. This shows RPT-1 is pattern-matching from the available context rather than memorizing the Titanic dataset from training.

Again, this is reassuring, but 98% accuracy for "Survived" is still too high to be real, especially with only minimal unmasked entries in the context window.

So if RPT-1 isn't memorizing the dataset, why is it so accurate on survival predictions? Let's test the ticket number an alternative field to predict. It is also a unique identifier, but it is not as "random" as the name because families were travelling together, for example.

## Predicting Ticket Numbers

In [126]:
random_masked = mask_random_entries(400, training_data, column_name='Ticket')
random_preds = get_rpt_predictions(random_masked, model, column_name='Ticket')
print(calculate_accuracy(training_data, random_masked, random_preds, target_column='Ticket'))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
0.2325


In [127]:
# Check if predicted tickets appear in the unmasked context
masked_df, unmasked_df = separate_masked_unmasked(random_masked, column_name='Ticket')
context_tickets = set(unmasked_df['Ticket'].values)

predicted_tickets = [p['Ticket'][0]['prediction'] for p in random_preds]
from_context = sum(1 for t in predicted_tickets if t in context_tickets)

print(f"Predictions from context tickets: {from_context}/{len(predicted_tickets)} = {from_context/len(predicted_tickets)*100:.1f}%")

Predictions from context tickets: 400/400 = 100.0%


While I have no benchmark for scoring 25% on this task, it sounds plaubible that RPT-1 could predict the ticket numbers from the context at this accuracy (as opposed to 98%). But it still does not explain why it is scoring so high on the "Survived" column.

Looking at the prediction results so far, here's a theory that I want to test in the next section.

## Testing a Theory

Could it be that RPT-1 is memorizing the Titanic dataset from training, and the binary nature of 'Survived' makes it easy to regurgitate correct answers as long as both possible values appear in context?

To verify this approach, we will do the following: Do 3 runs with only 2 unmaked rows:
- First run: Contains 2 unmasked rows, one containing a 1 and one containing 0
- Second Run: Contains 2 unmasked rows, both containing a 1
- Third run: Contains 2 unmasked rows, both containing a 0

I would expect the first run to score a very high, 97-98%. The second and the third run should predict only the value which was in the context, therefore the accuracy should be much lower.



In [128]:
def show_prediction_distribution(predictions, target_column='Survived'):
    """Display the distribution of predicted values."""
    predicted_values = [p[target_column][0]['prediction'] for p in predictions]
    from collections import Counter
    prediction_counts = Counter(predicted_values)
    
    print(f"Prediction distribution: {dict(prediction_counts)}")
    print(f"\nTotal predictions: {len(predicted_values)}")
    for value, count in prediction_counts.most_common():
        print(f"Predicted '{value}': {count}")
    
    return prediction_counts

In [129]:
masked_training_data = mask_last_entries(889, training_data)
masked_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,[PREDICT],3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,[PREDICT],1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,[PREDICT],3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [130]:
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate li

In [131]:
show_prediction_distribution(preds)

Prediction distribution: {'0': 476, '1': 413}

Total predictions: 889
Predicted '0': 476
Predicted '1': 413


Counter({'0': 476, '1': 413})

In [132]:
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [133]:
def mask_except_passengers(df, keep_passenger_ids, column_name='Survived', mask='[PREDICT]'):
    masked_df = df.copy()
    mask_condition = ~masked_df['PassengerId'].isin(keep_passenger_ids)
    masked_df.loc[mask_condition, column_name] = mask
    return masked_df

In [134]:
# Keep only passengers 1 and 2 unmasked
masked_training_data = mask_except_passengers(training_data, keep_passenger_ids=[2, 3])
masked_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,[PREDICT],3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,[PREDICT],1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,[PREDICT],3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [135]:
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
0.38245219347581555


In [136]:
show_prediction_distribution(preds)

Prediction distribution: {'1': 889}

Total predictions: 889
Predicted '1': 889


Counter({'1': 889})

In [137]:
# Keep only passengers 1 and 2 unmasked
masked_training_data = mask_except_passengers(training_data, keep_passenger_ids=[1, 5])
masked_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,[PREDICT],1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,[PREDICT],3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,[PREDICT],1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [138]:
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate li

In [139]:
show_prediction_distribution(preds)

Prediction distribution: {'0': 889}

Total predictions: 889
Predicted '0': 889


Counter({'0': 889})

My theory is partly confirmed and partly rejected: What we could observe is that also for "survived" RPT-1 only uses the values which are in the context window. Therefore, if we limit the information in the context window too much, and not all of the values which need to be predicted are in the context window, the prediction result deteriorates, or become somewhat random.

We could not observe the regurgitation of the Titanic data set which RPT-1 presumably has seen during training, but the overall quality remains to go to be true. To further proof this point, let's run two more experiments with only 10 unmasked entries and 20 unmasked entries:

In [140]:
masked_training_data = mask_last_entries(881, training_data)
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate li

In [141]:
masked_training_data = mask_last_entries(871, training_data)
preds = get_rpt_predictions(masked_training_data, model)
print(calculate_accuracy(training_data, masked_training_data, preds))

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
0.8576349024110218


With only very few additional unmasked entries in the context window, the prediction quality improve significantly. Without overly stressing the point, it improves too much.

Let's create a quick benchmark, using a traditional Gradient Boosting model, to rule out that there are patterns in the dataset which make predictions just on the training set more easy than on the test set.

## Training Gradient Boosting as a Benchmark

Let's create a quick Gradient Boosting baseline to see if RPT-1's 98% accuracy is plausible or if something strange is happening.

First, with a random train/test split:

In [142]:
#!pip install scikit-learn

In [143]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Prepare data (simple encoding)
train_df = training_data.copy()
train_df['Sex'] = LabelEncoder().fit_transform(train_df['Sex'])
train_df['Embarked'] = train_df['Embarked'].fillna('S')
train_df['Embarked'] = LabelEncoder().fit_transform(train_df['Embarked'])
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
X = train_df[features]
y = train_df['Survived'].astype(int)

# Same split: 491 train, 400 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=400, random_state=42)

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train, y_train)
print(f"Gradient Boosting accuracy: {clf.score(X_test, y_test):.1%}")

Gradient Boosting accuracy: 79.5%


That's more realistic: 80% is what we'd expect. But maybe the last 400 rows are just easier to predict? Let's test with the same split RPT-1 used:

In [144]:
# Use the EXACT same split for both methods
# Test on the last 400 rows like you did with RPT-1
X_train = X.iloc[:491]
X_test = X.iloc[491:]
y_train = y.iloc[:491]
y_test = y.iloc[491:]

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train, y_train)
print(f"Gradient Boosting accuracy (last 400): {clf.score(X_test, y_test):.1%}")

Gradient Boosting accuracy (last 400): 82.2%


Gradient Boosting gets 82% on the same split: Better than random, but nowhere near RPT-1's 98%. This confirms something unusual is happening with RPT-1's predictions on the training set.

The mystery remains: Why is RPT-1 scoring so unrealistically high? Rather than speculate further, let's move to what really matters: Predictions on the actual test set that we'll submit to Kaggle.

## Full Titanic Dataset with RPT-1

Despite the unexplained 98% accuracy on masked training data, let's see how RPT-1 performs on the actual test set where we can't verify predictions ourselves.

To work on the full Titanic data set from Kaggle, we need to add the mask column to the test set and combine the training and test set.

Afterwards, we can run the predictions and [submit the results](https://www.kaggle.com/competitions/titanic/) to Kaggle.


In [145]:
train_df = pd.read_csv("train.csv")
train_df['Survived'] = train_df['Survived'].astype(str)
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [146]:
test_df = pd.read_csv("test.csv")
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [147]:
test_df["Survived"] = '[PREDICT]'
combined_df = pd.concat([train_df, test_df], ignore_index=True)
combined_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,[PREDICT],3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,[PREDICT],1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,[PREDICT],3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,[PREDICT],3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [148]:
def save_preds(preds, filename="results.csv"):
    res = [(pred["PassengerId"], int(pred["Survived"][0]["prediction"])) for pred in preds]
    df = pd.DataFrame(res, columns=["PassengerId", "Survived"])
    df.to_csv(filename, index=False)

In [149]:
# Get predictions for the test set
test_preds = get_rpt_predictions(combined_df, model)
print(f"Got {len(test_preds)} predictions")

# Save submission file
save_preds(test_preds, filename=f"submit_aicore-rpt-1-{model_size}-no-feature-engineering.csv")

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Got 418 predictions


The result for the submitted file is 0.76555, matching the gender-based baseline. This is much more realistic than the 98% we saw on the training set, but there's clearly room for improvement. Let's see if we can help RPT-1 with some feature engineering.

## Full Titanic Dataset with Feature Engineering with RPT-1

Let's do some feature engineering to see if we can improve the result. A typocal recommendation is to extract the title as a separate column, so let's start there.

In [150]:
# Feature engineering - add Title to both train and test
def add_title_feature(df):
    df = df.copy()
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')
    # Consolidate rare titles
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    return df

# Apply to both datasets
train_df_fe = add_title_feature(train_df)
test_df_fe = add_title_feature(test_df)

# Combine
combined_df_fe = pd.concat([train_df_fe, test_df_fe], ignore_index=True)

print("Titles in data:")
print(combined_df_fe['Title'].value_counts())

Titles in data:
Title
Mr        757
Miss      264
Mrs       198
Master     61
Rare       29
Name: count, dtype: int64


In [151]:
combined_df_fe.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [152]:
# Get predictions with feature engineering
test_preds_fe = get_rpt_predictions(combined_df_fe, model)
print(f"Got {len(test_preds_fe)} predictions")

# Save submission
save_preds(test_preds_fe, filename=f"submit_aicore-rpt-1-{model_size}-with-title.csv")

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Got 418 predictions


Surprisingly, the score dropped to 0.76076. Let's see if this is only relared to this one feature, or of a broader pattern emerges.

In [153]:
def add_features(df):
    df = df.copy()
    
    # Title
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
    # Family
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # Age bins (fill missing with median first)
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['AgeBin'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 60, 100], labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])
    
    # Deck from Cabin
    df['Deck'] = df['Cabin'].str[0].fillna('Unknown')
    
    # Fare per person
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    
    return df

# Apply
train_df_fe2 = add_features(train_df)
test_df_fe2 = add_features(test_df)
combined_df_fe2 = pd.concat([train_df_fe2, test_df_fe2], ignore_index=True)

combined_df_fe2[['Name', 'Title', 'FamilySize', 'IsAlone', 'AgeBin', 'Deck']].head(10)

,Name,Title,FamilySize,IsAlone,AgeBin,Deck
0,"Braund, Mr. Owen Harris",Mr,2,0,Adult,Unknown
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs,2,0,Adult,C
2,"Heikkinen, Miss. Laina",Miss,1,1,Adult,Unknown
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs,2,0,Adult,C
4,"Allen, Mr. William Henry",Mr,1,1,Adult,Unknown
5,"Moran, Mr. James",Mr,1,1,Adult,Unknown
6,"McCarthy, Mr. Timothy J",Mr,1,1,Middle,E
7,"Palsson, Master. Gosta Leonard",Master,5,0,Child,Unknown
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",Mrs,3,0,Adult,Unknown
9,"Nasser, Mrs. Nicholas (Adele Achem)",Mrs,2,0,Teen,Unknown


In [154]:
combined_df_fe2.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone,AgeBin,Deck,FarePerPerson
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2,0,Adult,Unknown,3.62500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2,0,Adult,C,35.64165
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1,Adult,Unknown,7.92500
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2,0,Adult,C,26.55000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1,1,Adult,Unknown,8.05000
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q,Mr,1,1,Adult,Unknown,8.45830
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr,1,1,Middle,E,51.86250
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master,5,0,Child,Unknown,4.21500
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs,3,0,Adult,Unknown,3.71110
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs,2,0,Teen,Unknown,15.03540


In [155]:
combined_df_fe2.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone,AgeBin,Deck,FarePerPerson
1299,1300,[PREDICT],3,"Riordan, Miss. Johanna Hannah""""",female,27.0,0,0,334915,7.7208,NaN,Q,Miss,1,1,Adult,Unknown,7.720800
1300,1301,[PREDICT],3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S,Miss,3,0,Child,Unknown,4.591667
1301,1302,[PREDICT],3,"Naughton, Miss. Hannah",female,27.0,0,0,365237,7.7500,NaN,Q,Miss,1,1,Adult,Unknown,7.750000
1302,1303,[PREDICT],1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q,Mrs,2,0,Adult,C,45.000000
1303,1304,[PREDICT],3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S,Miss,1,1,Adult,Unknown,7.775000
1304,1305,[PREDICT],3,"Spector, Mr. Woolf",male,27.0,0,0,A.5. 3236,8.0500,NaN,S,Mr,1,1,Adult,Unknown,8.050000
1305,1306,[PREDICT],1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Rare,1,1,Adult,C,108.900000
1306,1307,[PREDICT],3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,1,1,Adult,Unknown,7.250000
1307,1308,[PREDICT],3,"Ware, Mr. Frederick",male,27.0,0,0,359309,8.0500,NaN,S,Mr,1,1,Adult,Unknown,8.050000
1308,1309,[PREDICT],3,"Peter, Master. Michael J",male,27.0,1,1,2668,22.3583,NaN,C,Master,3,0,Adult,Unknown,7.452767


In [156]:
# Get predictions with full feature engineering
test_preds_fe2 = get_rpt_predictions(combined_df_fe2, model)
print(f"Got {len(test_preds_fe2)} predictions")

# Save submission
save_preds(test_preds_fe2, filename=f"submit_aicore-rpt-1-{model_size}-with-full-fe.csv")

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Got 418 predictions


The score only slightly increased to 0.76794. It seems like the common feature engineering only has minimal impact, maybe RPT-1 is already extracting these patterns from the raw data?

To see how good/bad this result actually is, in the next section, we'll compare this to the performence of Gradient Boosting models with the same data, after all RPT-1's claim to fame is that it can replace traditional machine learning models which need to be trained individually.

## Gradient Boosting Baseline with Feature Engineering

Let's train a Gradient Boosting model using the same engineered features we gave to RPT-1:

In [157]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# Prepare the feature-engineered data for Gradient Boosting
train_gb = combined_df_fe2[combined_df_fe2['Survived'] != '[PREDICT]'].copy()
test_gb = combined_df_fe2[combined_df_fe2['Survived'] == '[PREDICT]'].copy()

# Encode categorical columns
label_encoders = {}
categorical_cols = ['Sex', 'Embarked', 'Title', 'AgeBin', 'Deck']

for col in categorical_cols:
    le = LabelEncoder()
    # Fit on combined data to handle all categories
    combined_values = combined_df_fe2[col].astype(str).fillna('Unknown')
    le.fit(combined_values)
    train_gb[col] = le.transform(train_gb[col].astype(str).fillna('Unknown'))
    test_gb[col] = le.transform(test_gb[col].astype(str).fillna('Unknown'))
    label_encoders[col] = le

# Fill remaining NaN values
train_gb['Fare'] = train_gb['Fare'].fillna(train_gb['Fare'].median())
test_gb['Fare'] = test_gb['Fare'].fillna(test_gb['Fare'].median())
test_gb['FarePerPerson'] = test_gb['FarePerPerson'].fillna(test_gb['FarePerPerson'].median())

# Features to use
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
            'Title', 'FamilySize', 'IsAlone', 'AgeBin', 'Deck', 'FarePerPerson']

X_train = train_gb[features]
y_train = train_gb['Survived'].astype(int)
X_test = test_gb[features]

# Train Gradient Boosting
clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict
predictions = clf.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test_gb['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submit_gradientboosting_full_fe.csv', index=False)

print(f"Training accuracy: {clf.score(X_train, y_train):.1%}")
print(f"Predictions saved to submit_gradientboosting_full_fe.csv")
print(submission.head(10))

Training accuracy: 91.0%
Predictions saved to submit_gradientboosting_full_fe.csv
     PassengerId  Survived
891          892         0
892          893         0
893          894         0
894          895         0
895          896         1
896          897         0
897          898         0
898          899         0
899          900         1
900          901         0


This submission scored 0.76555. Both RPT-1 and Gradient Boosting achieve similar baseline performance (~76.5%), showing that RPT-1 can match traditional ML without any training. More advanced feature engineering could potentially improve both models further. Let's try another round of feature engineering in the next section.

## Even more Feature Engineering

Let's try a more sophisticated approach to feature engineering. Beyond the basic features we've already tried (Title, FamilySize, AgeBin, Deck), we'll add some features that capture group dynamics and social context:

- Family and Ticket Group Survival Rates: Calculate the survival rate of other passengers sharing the same surname or ticket number (using only training data to avoid leakage)
- Family Size Categories: Group families into Single, Small (2-4), and Large (5+) 
- Fare Bins: Categorize fares into quartiles
- Is Mother: Identify married women (Mrs) traveling with children
- Is Child with Family: Flag children under 16 traveling with family members

The key insight here is that survival wasn't just individual, but families and traveling groups often survived or perished together. By encoding these group patterns, we might help the model capture social dynamics that simple demographic features miss.

In [158]:
def add_advanced_features(train_df, test_df):
    """Add advanced features including family/ticket survival rates."""
    
    # Combine for consistent processing
    train_df = train_df.copy()
    test_df = test_df.copy()
    train_df['IsTrainSet'] = True
    test_df['IsTrainSet'] = False
    combined = pd.concat([train_df, test_df], ignore_index=True)
    
    # Basic features (from before)
    combined['Title'] = combined['Name'].str.extract(r' ([A-Za-z]+)\.')
    combined['Title'] = combined['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    combined['Title'] = combined['Title'].replace(['Mlle', 'Ms'], 'Miss')
    combined['Title'] = combined['Title'].replace('Mme', 'Mrs')
    
    combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1
    combined['IsAlone'] = (combined['FamilySize'] == 1).astype(int)
    
    # Family size categories (small families survived better)
    combined['FamilySizeCategory'] = combined['FamilySize'].apply(
        lambda x: 'Single' if x == 1 else ('Small' if x <= 4 else 'Large'))
    
    combined['Age'] = combined['Age'].fillna(combined.groupby('Title')['Age'].transform('median'))
    combined['AgeBin'] = pd.cut(combined['Age'], bins=[0, 12, 20, 40, 60, 100], 
                                 labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])
    
    combined['Deck'] = combined['Cabin'].str[0].fillna('Unknown')
    
    combined['Fare'] = combined['Fare'].fillna(combined['Fare'].median())
    combined['FarePerPerson'] = combined['Fare'] / combined['FamilySize']
    combined['FareBin'] = pd.qcut(combined['Fare'], 4, labels=['Low', 'Medium', 'High', 'VeryHigh'])
    
    # --- ADVANCED FEATURES ---
    
    # 1. Surname for family grouping
    combined['Surname'] = combined['Name'].str.split(',').str[0]
    
    # 2. Family Survival Rate (from training data only!)
    train_data = combined[combined['IsTrainSet'] == True].copy()
    train_data['Survived_int'] = train_data['Survived'].astype(int)
    
    # Group families and calculate survival rate
    family_survival = train_data.groupby('Surname')['Survived_int'].agg(['mean', 'count'])
    family_survival.columns = ['FamilySurvivalRate', 'FamilyCount']
    # Only use families with 2+ members for reliability
    family_survival.loc[family_survival['FamilyCount'] < 2, 'FamilySurvivalRate'] = 0.5
    combined = combined.merge(family_survival[['FamilySurvivalRate']], left_on='Surname', right_index=True, how='left')
    combined['FamilySurvivalRate'] = combined['FamilySurvivalRate'].fillna(0.5)
    
    # 3. Ticket Survival Rate
    ticket_survival = train_data.groupby('Ticket')['Survived_int'].agg(['mean', 'count'])
    ticket_survival.columns = ['TicketSurvivalRate', 'TicketCount']
    ticket_survival.loc[ticket_survival['TicketCount'] < 2, 'TicketSurvivalRate'] = 0.5
    combined = combined.merge(ticket_survival[['TicketSurvivalRate']], left_on='Ticket', right_index=True, how='left')
    combined['TicketSurvivalRate'] = combined['TicketSurvivalRate'].fillna(0.5)
    
    # 4. Combined survival indicator (average of family and ticket)
    combined['GroupSurvivalRate'] = (combined['FamilySurvivalRate'] + combined['TicketSurvivalRate']) / 2
    
    # 5. Is Mother (Mrs with Parch > 0)
    combined['IsMother'] = ((combined['Title'] == 'Mrs') & (combined['Parch'] > 0)).astype(int)
    
    # 6. Is Child with Family
    combined['IsChildWithFamily'] = ((combined['Age'] < 16) & (combined['FamilySize'] > 1)).astype(int)
    
    # Split back
    train_result = combined[combined['IsTrainSet'] == True].drop('IsTrainSet', axis=1)
    test_result = combined[combined['IsTrainSet'] == False].drop('IsTrainSet', axis=1)
    
    # Remove helper columns
    if 'Survived_int' in train_result.columns:
        train_result = train_result.drop('Survived_int', axis=1)
    
    return train_result, test_result

# Apply advanced features
train_df_adv, test_df_adv = add_advanced_features(train_df, test_df)

# Combine for RPT-1
combined_df_adv = pd.concat([train_df_adv, test_df_adv], ignore_index=True)

print("New columns:", [c for c in combined_df_adv.columns if c not in combined_df.columns])
print(f"\nSample of advanced features:")
print(combined_df_adv[['Name', 'Surname', 'FamilySurvivalRate', 'TicketSurvivalRate', 'GroupSurvivalRate', 'IsMother', 'IsChildWithFamily']].head(10))

New columns: ['Title', 'FamilySize', 'IsAlone', 'FamilySizeCategory', 'AgeBin', 'Deck', 'FarePerPerson', 'FareBin', 'Surname', 'FamilySurvivalRate', 'TicketSurvivalRate', 'GroupSurvivalRate', 'IsMother', 'IsChildWithFamily']

Sample of advanced features:
                                                Name    Surname  \
0                            Braund, Mr. Owen Harris     Braund   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    Cumings   
2                             Heikkinen, Miss. Laina  Heikkinen   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   Futrelle   
4                           Allen, Mr. William Henry      Allen   
5                                   Moran, Mr. James      Moran   
6                            McCarthy, Mr. Timothy J   McCarthy   
7                     Palsson, Master. Gosta Leonard    Palsson   
8  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)    Johnson   
9                Nasser, Mrs. Nicholas (Adele Achem)     Nasser   

   Fami

In [159]:
# Get predictions with advanced features
test_preds_adv = get_rpt_predictions(combined_df_adv, model)
print(f"Got {len(test_preds_adv)} predictions")

# Save submission
save_preds(test_preds_adv, filename=f"submit_aicore-rpt-1-{model_size}-advanced-fe.csv")

Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Rate limited. Waiting 1 seconds before retry 1/7...
Rate limited. Waiting 2 seconds before retry 2/7...
Rate limited. Waiting 4 seconds before retry 3/7...
Rate limited. Waiting 8 seconds before retry 4/7...
Rate limited. Waiting 16 seconds before retry 5/7...
Rate limited. Waiting 32 seconds before retry 6/7...
Got 418 predictions


This submission scored 0.78229, scoring significantly higher than the previous one. This shows that feature engineering can improve RPT-1's performance, just like in traditional ML.

Let's compare to the traditional gradient boosting approach:

In [160]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# Prepare the advanced feature-engineered data for XGBoost
train_gb_adv = combined_df_adv[combined_df_adv['Survived'] != '[PREDICT]'].copy()
test_gb_adv = combined_df_adv[combined_df_adv['Survived'] == '[PREDICT]'].copy()

# Encode categorical columns
categorical_cols = ['Sex', 'Embarked', 'Title', 'AgeBin', 'Deck', 'FamilySizeCategory', 'FareBin']

for col in categorical_cols:
    le = LabelEncoder()
    combined_values = combined_df_adv[col].astype(str).fillna('Unknown')
    le.fit(combined_values)
    train_gb_adv[col] = le.transform(train_gb_adv[col].astype(str).fillna('Unknown'))
    test_gb_adv[col] = le.transform(test_gb_adv[col].astype(str).fillna('Unknown'))

# Features to use (all the good ones)
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
            'Title', 'FamilySize', 'IsAlone', 'FamilySizeCategory', 'AgeBin', 'Deck', 
            'FarePerPerson', 'FareBin', 'FamilySurvivalRate', 'TicketSurvivalRate', 
            'GroupSurvivalRate', 'IsMother', 'IsChildWithFamily']

X_train = train_gb_adv[features]
y_train = train_gb_adv['Survived'].astype(int)
X_test = test_gb_adv[features]

# Train Gradient Booting with some tuning
clf = GradientBoostingClassifier(
    n_estimators=150,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)
clf.fit(X_train, y_train)

# Predict
predictions = clf.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test_gb_adv['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submit_gradientboosting_advanced_fe.csv', index=False)

print(f"Training accuracy: {clf.score(X_train, y_train):.1%}")
print(f"Predictions saved to submit_gradientboosting_advanced_fe.csv")

# Feature importance
import numpy as np
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)
print("\nTop 10 features:")
print(feature_importance.head(10))

Training accuracy: 96.6%
Predictions saved to submit_gradientboosting_advanced_fe.csv

Top 10 features:
               feature  importance
1                  Sex    0.328089
17   GroupSurvivalRate    0.298040
5                 Fare    0.084999
13       FarePerPerson    0.079934
2                  Age    0.067069
16  TicketSurvivalRate    0.066718
7                Title    0.018501
12                Deck    0.011158
19   IsChildWithFamily    0.009703
6             Embarked    0.007782


Interesting, the Gradient Boosting submission only scored 0.76076. With all these features it actually performed worse. This is a classic case of overfitting. Let's try a more robust approach with cross validation.

In [161]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Simpler feature set - remove redundant features
features_simple = ['Pclass', 'Sex', 'Age', 'Fare', 'Title', 'FamilySize', 
                   'FamilySurvivalRate', 'TicketSurvivalRate', 'IsMother', 'IsChildWithFamily']

X_train = train_gb_adv[features_simple]
y_train = train_gb_adv['Survived'].astype(int)
X_test = test_gb_adv[features_simple]

# More regularized model to prevent overfitting
clf = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=3,  # Reduced from 4
    learning_rate=0.05,  # Reduced from 0.1
    min_samples_leaf=10,  # Add minimum samples
    max_features='sqrt',  # Limit features per split
    random_state=42
)

# Check cross-validation score first
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
print(f"CV scores: {cv_scores}")
print(f"Mean CV score: {cv_scores.mean():.3f} (+/- {cv_scores.std()*2:.3f})")

clf.fit(X_train, y_train)
print(f"Training accuracy: {clf.score(X_train, y_train):.1%}")

# Predict and save
predictions = clf.predict(X_test)
submission = pd.DataFrame({
    'PassengerId': test_gb_adv['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submit_gradientboosting_advanced_fe_v2.csv', index=False)
print("Saved to submit_gradientboosting_advanced_fe_v2.csv")

CV scores: [0.90502793 0.86516854 0.88202247 0.87078652 0.90449438]
Mean CV score: 0.885 (+/- 0.033)
Training accuracy: 89.3%
Saved to submit_gradientboosting_advanced_fe_v2.csv


This submission scored 0.77751, which is a lot better then the overfitted version. The advanced feature engineering (finally) helped both RPT-1 and Gradient Boosting.

If we zoom out a bit, we can see the benefits of using RPT-1. With RPT-1 we could directly benefit from the feature engineering without having to train and iterate in a new model. This is a huge advantage, especially if you are not a data scientist or machine learning expert.

## Conclusion

In this notebook we explored SAP's RPT-1 model on the Titanic challenge. Here's what we learned:

Early experiments showed RPT-1 achieving unrealistic 98% accuracy when predicting masked entries in the training set. Why exactly it could perform so well remains a mystery. We could rule out that RPT-1 memorized the dataset, but the mystery of why it scores so unrealistically well on masked training data remains unsolved. If you have a possible explanation, please let me know.

On the actual test set, RPT-1 scored 0.76555 without feature engineering, matching the gender-based baseline and traditional Gradient Boosting models. Adding advanced features (family/ticket survival rates, social context) improved RPT-1 to 0.78229. The same features helped Gradient Boosting reach 0.77751 (after avoiding overfitting). This performance confirms SAP's claim that RPT-1 can replace classical machine learning models without training.

In summary, RPT-1 performed very well. The results are solid, considering that many models in the leaderboard struggle to get to the baseline performance. RPT-1 reached this level without any data cleansing or feature engineering. In addition, you can see how RPT-1 can improve iteration speed. After engineering new features, we could immediately test without retraining or changing the model architecture. For Gradient Boosting, we had to train multiple models and use cross-validation to avoid overfitting. With RPT-1 this was not necessary which would be huge if this finding would carry over to other use cases as well.

So far I am very impressed with RPT-1. I am looking forward to exploring more use cases and seeing how it performs in more complex scenarios.